In [ ]:
!gshell init

In [ ]:
!gshell info --with-id 17axH-dYh-Wss_ec92ugSl8AmSuZS-xn9

In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.4'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [6]:
key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
for x in key_value[5:]:
    print(123)


In [12]:
parser_version = '1.6.4'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "ДД по практикам\Практика недропользования и экологии\1. ДС N 8_испр.docx"

{"documents":[{"documentType":"SUPPLEMENTARY_AGREEMENT","paragraphs":[{"paragraphHeader":{"offset":0,"text":"Дополнительное соглашение №8\r\nк договору подряда\r\nот 10.01.2020 года №ГНЯ-20/33/00041/Р/6-3  \r\n","length":95},"paragraphBody":{"offset":89,"text":"г. Тюмень\r\n03.09.2021г.\r\nОбщество с ограниченной ответственностью «Газпромнефть – Ямал»                                   (ООО «Газпромнефть – Ямал»), именуемое в дальнейшем - Заказчик, в лице директора Федотова Виталия Анатольевича, действующего на основании Устава, с одной стороны, и\r\nОбщество с ограниченной ответственностью «Защита» (ООО «Защита»), именуемое в дальнейшем  - Подрядчик, в лице генерального директора Сасова Ильи Владимировича, действующего на основании Устава, с другой стороны, \r\nсовместно именуемые  - Стороны, а отдельно также – Сторона, Стороны заключили настоящее дополнительное соглашение к договору строительного подряда от 10.01.2020 года №ГНЯ-20/33/00041/Р/6-3 (далее по тексту – Договор) о нижеследу

In [ ]:
import subprocess

s = [
    "gshell",
    "init"
]

subprocess.run(s, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

In [11]:
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import re

index = 1
number_of_docs = 0
parser_version = '1.6.4'
root = "ДД по практикам"

arrOfAllDocs = []
result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

paragraph_number = 1
s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

for root, dirnames, filenames in os.walk('ДД по практикам'):
    if len(root.split('\\')) == 1: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))

print(
    "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")


def find_let(document,documentType = None):
    key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
    if documentType == 'SUPPLEMENTARY_AGREEMENT':
        key_value.append('в следующей редакции:')
        key_value.append('заключили настоящее Дополнительное соглашение к Договору.')
        key_value.append('редакции:')
    result = ""
    for i, p in enumerate(document['paragraphs']):
        if any(f.lower() in p['paragraphBody']['text'].lower() or f.lower() in
               p['paragraphHeader']['text'].lower() for f in
               key_value[:4]) or any(f.lower() in p['paragraphBody']['text'].lower() or f.lower() in
                                     p['paragraphHeader']['text'].lower() for f in
                                     key_value[5:]):
            text = ""
            for x in key_value[:4]:
                if x.lower() in p['paragraphBody']['text'].lower():
                    text += p['paragraphBody']['text'].split(x)[1]
                    break
                if x.lower() in p['paragraphHeader']['text'].lower():
                    text += p['paragraphBody']['text']
                    break

            if text == "":
                for x in key_value[5:]:
                    if x.lower() in p['paragraphBody']['text'].lower():
                        text += p['paragraphBody']['text'].split(x)[1]
                        break
                    if x.lower() in p['paragraphHeader']['text'].lower():
                        text += p['paragraphBody']['text']
                        break

            if text == "": continue

            # text += "".join(
            #     str(x['paragraphBody']['text']) if re.match("^ *\d?[.]", str(
            #         x['paragraphBody']['text'])) or re.match("^ *\d?[.]", str(
            #         x['paragraphHeader']['text'])) else '' for x in
            #     document['paragraphs'][i:i + 4])
            #
            # textHeader = p['paragraphHeader']['text'] + "\n".join(
            #     str(x['paragraphHeader']['text']) if re.match("^ *\d?[.] ", str(
            #         x['paragraphBody']['text'])) or re.match("^ *\d?[.] ", str(
            #         x['paragraphHeader']['text'])) else '' for x in document['paragraphs'])

            text += "".join(
                str(x['paragraphBody']['text']) for x in document['paragraphs'][i+1:i + 4])

            textHeader = p['paragraphHeader']['text'] + "\n".join(
                str(x['paragraphHeader']['text']) for x in document['paragraphs'][i:i + 4])


            result = {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": p['paragraphBody']['offset'],
                "text": text,
                "length": len(text),
                "offsetHeader": p['paragraphHeader']['offset'],
                "textHeader": textHeader,
                "lengthHeader": len(textHeader)
            }
            break
    return result


time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find("Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"при конвертации в base64, исключение = {e.msg}")
        print("=" * 200)
        continue

    response = requests.post(
        "http://localhost:8083/document-parser",
        data=json.dumps({
            "base64Content": encoded_string,
            "documentFileType": docs.split(".")[-1].upper()
        }),
        headers=headers
    )
    resArr = []
    try:
        resArr = response.json()['documents']
    except Exception as e:
        print(f"\nОшибка в файле {docs}")
        print(f"Ответ от парсера {response.json()}")
        print(f"Исключение = {e}")
        print("=" * 200)
        continue

    sys.stdout.write(f"\rПроверка документа под номером {index} и ИД {len(result)}")
    sys.stdout.flush()
    index += 1

    document = []
    if resArr:
        document = resArr[0]
    else:
        continue

    if document['documentType'] == "CONTRACT":
        keys = ['предмет договра','предмет договора','предмет контракта', 'Общие состояние', 'Общие положение','Статья 1']
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(z.lower() in re.sub(' +', ' ', p['paragraphHeader']['text'].lower()) for z in keys) and p['paragraphBody']['length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue
        for i, p in enumerate(document['paragraphs']):
            if any(z.lower() in p['paragraphBody']['text'].lower() for z in [
                'предмет договра','предмет договора','предмет контракта', 'Общие состояние', 'Общие положение','Статья 1']) and p['paragraphBody']['length'] > 20:
                text=""

                for key in keys:
                    if key.lower() in re.sub(' +', ' ', p['paragraphBody']['text']).lower():
                        array_of_text = re.split(f"(?i)({key})",re.sub(' +', ' ', p['paragraphBody']['text']))
                        end_text = 0
                        try:
                            last_symbol = re.search("\s\d[ .]\d?[\s|\u00A0|.]*$",array_of_text[0])
                            if last_symbol:
                                end_text = int(last_symbol.group().replace(" ","").split(".")[0])
                        except ValueError as ex:
                            print(f"cannot converted str to int")
                            text = array_of_text[2]
                            break

                        if end_text:
                            end_text += 1
                            print("\nEnd = ",end_text)
                            text = re.split(f"\s({end_text})[. ]",array_of_text[2])[0]
                            break

                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": text,
                        "length": len(text),
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue
        obj = find_let(document)
        if obj != "":
            result_of_possible2.append(obj)
            flag = True

        if flag: continue
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "\n+++++++++++++\n".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "SUPPLEMENTARY_AGREEMENT":
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(f.lower() in p['paragraphHeader']['text'].lower() for f in
                   ['Статья 1']) and p['paragraphBody'][
                'length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue
        obj = find_let(document,document['documentType'])
        if obj != "":
            result_of_possible2.append(obj)
            flag = True

        if flag: continue
        #document['paragraphs'][0]['paragraphHeader']['text']
        result_of_possible.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": document['paragraphs'][0]['paragraphBody']['offset'],
                "text": "".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in document['paragraphs']),
                "offsetHeader": document['paragraphs'][0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })
    elif document['documentType'] == "AGREEMENT":
        flag = False
        for i, p in enumerate(document['paragraphs']):
            if any(f.lower() in re.sub(' +', ' ', p['paragraphHeader']['text'].lower()) for f in
                   ['Предмет соглашения', 'Общие состоян', 'Общие положение','Статья 1']) and p['paragraphBody'][
                'length'] > 20:
                result.append(
                    {
                        "path": docs,
                        "name": docs.split("\\")[-1],
                        "documentType": document['documentType'],
                        "offset": p['paragraphBody']['offset'],
                        "text": p['paragraphBody']['text'],
                        "length": p['paragraphBody']['length'],
                        "offsetHeader": p['paragraphHeader']['offset'],
                        "textHeader": p['paragraphHeader']['text'],
                        "lengthHeader": p['paragraphHeader']['length']
                    })
                flag = True
                break
        if flag: continue

        obj = find_let(document)
        if obj != "":
            result_of_possible2.append(obj)
            flag = True

        if flag: continue

        # arr_of_paragraphs[
        #     paragraph_number if paragraph_number < len(arr_of_paragraphs) else 0][
        #     'paragraphBody']['text'] if
        # arr_of_paragraphs[paragraph_number if paragraph_number < len(
        #     arr_of_paragraphs) else 0]['paragraphBody']['length'] > 20 else "".join(
        #     str(x['paragraphBody']['text']) for x in document['paragraphs'])

        arr_of_paragraphs = document['paragraphs']
        result_of_fail.append(
            {
                "path": docs,
                "name": docs.split("\\")[-1],
                "documentType": document['documentType'],
                "offset": arr_of_paragraphs[0]['paragraphBody']['offset'],
                "text": "\n+++++++++++++\n".join(
                    str(x['paragraphBody']['text']) for x in document['paragraphs']),
                "length": sum(i['paragraphBody']['length'] for i in arr_of_paragraphs),
                "offsetHeader": arr_of_paragraphs[0]['paragraphHeader']['offset'],
                "textHeader": "\n+++++++++++++\n".join(
                    str(x['paragraphHeader']['text']) for x in document['paragraphs']),
                "lengthHeader": sum(i['paragraphHeader']['length'] for i in document['paragraphs'])
            })

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа, убейте в ручную процесс java')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible)
df.to_excel(writer, 'SO SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible2)
df.to_excel(writer, 'О ниже', engine='xlsxwriter')
sheets_bad = writer.sheets['О ниже']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")

Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения #31
Готово
Запустился успешно
Общее количество документов = 1261
Проверка документа под номером 244 и ИД 99
End =  2
Проверка документа под номером 362 и ИД 155
Ошибка в файле ДД по практикам\Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом\Дополнительное соглашение N1 Г-РП.doc
Ответ от парсера {'timestamp': '2021-12-15T15:40:29.775+0000', 'status': 400, 'error': 'Bad Request', 'message': 'The document is really a OOXML file', 'path': '/document-parser', 'version': '1.6.4'}
Исключение = 'documents'
Проверка документа под номером 380 и ИД 161
Ошибка в файле ДД по практикам\Практика правового сопровождения операционной деятельности БРД\Договоры\04-6.docx
Ответ от парсера {'timestamp': '2021-12-15T15:40:30.670+0000', 'status': 500, 'error': 'Internal Server Error', 'message': 'Cannot invoke "java.math.BigInteger.intValue()" because "abstra

In [14]:
!java --version

java 17 2021-09-14 LTS
Java(TM) SE Runtime Environment (build 17+35-LTS-2724)
Java HotSpot(TM) 64-Bit Server VM (build 17+35-LTS-2724, mixed mode, sharing)
